# Mask R-CNN - Inspect Custom Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [ ]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import custom 

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

custom_WEIGHTS_PATH = "/path/to/.h5"  # TODO: update the path of trained model, it's in log folder. 

print(MODEL_DIR)

## Configurations

In [5]:
config = custom.DataConfig()
custom_DIR = os.path.join(ROOT_DIR, "dataset")

In [7]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.2
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Notebook Preferences

In [8]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [9]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Validation Dataset

In [ ]:
# Load validation dataset
dataset = custom.CustomDataset()
dataset.load_dataset("/datasets/val/coco_instances.json", "/datasets/val/images")

# Must call before using the dataset
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))



## Load Model

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
# load the last model you trained
# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

In [ ]:
from importlib import reload 
reload(visualize)

# Run Detection on Synthetic images

In [17]:
def integers(a, b):
         return list(range(a, b+1))

In [ ]:
image_ids =  integers(0,len(dataset.image_ids)-1) # TODO: update len(dataset.image_ids)-1 
                                                         #to an integer that decides the upper limit of images to be visualized 
APs = []
precisionss = []
recallss = []

#inference_start = time.time()
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # Run object detection
    results = model.detect([image], verbose=1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'])

    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'],iou_threshold=0.5) #threshold = 0.5
    visualize.plot_precision_recall(AP, precisions, recalls)
    APs.append(AP)
    precisionss.append(precisions)
    recallss.append(recalls)
    


#inference_end = time.time()
#print('Inference Time: %0.2f Minutes'%((inference_end - inference_start)/60))
print("mAP: ", np.mean(APs))


## Use trained model to predict raw images

## Note: raw images have to be same width and height as synthetic images

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from matplotlib import pyplot
from matplotlib.patches import Rectangle
 
# draw an image with detected objects
def draw_image_with_boxes(filename, boxes_list):
     # load the image
     data = pyplot.imread(filename)
     # plot the image
     pyplot.imshow(data)
     # get the context for drawing boxes
     ax = pyplot.gca()
     # plot each box
     for box in boxes_list:
          # get coordinates
          y1, x1, y2, x2 = box
          # calculate width and height of the box
          width, height = x2 - x1, y2 - y1
          # create the shape
          rect = Rectangle((x1, y1), width, height, fill=False, color='red')
          # draw the box
          ax.add_patch(rect)
     # show the plot
     pyplot.show()

In [ ]:
from os import listdir
image_path = '/path/to/rawimages' ## TODO: update the path of raw images, if you want to predict real images instead of synthetic images 
for filename in listdir(image_path):
    img = load_img(image_path+'/'+filename)
    img = img_to_array(img)
    results = model.detect([img], verbose=0)
    draw_image_with_boxes(image_path+'/'+filename, results[0]['rois'])

# Show Statistics

## Note: test scores below are validated based on "synthetic" validation images. The scores might not be as accuate as using raw images in validation set. However, manually create annotations for raw, val set is labour-consuming. This limtation is metioned in my dissertation.

In [ ]:
# calculate precision and recall 
p = np.concatenate(precisionss, axis=None)
r = np.concatenate(recallss, axis=None)

In [ ]:
#plot simple ROC curve 
plt.plot(r, p, linewidth=1)

In [ ]:
# make a dataframe stores test scores
import pandas as pd
F1s = 2* (p*r) / (p+r)
df = pd.DataFrame({"precison": p, "recall": r, "F1":F1s})

In [ ]:
#print test scores: mAP, mPrecision, mRecall and mF1
print("mAP: ", np.mean(APs))
print("mPrecision: ", np.mean(p))
print("mRecall: ", np.mean(r))
print("mF1: ", np.mean(F1s[np.logical_not(np.isnan(F1s))]))

In [ ]:
#print distribution of Precision, Recall and F1
plt.hist(p)

In [ ]:
plt.hist(r)

In [ ]:
plt.hist(F1s)